In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Load dataframe from .pickle file
play_file_pickle = '/Users/Xiaoxi/Desktop/BitTiger/Capstone/data/output/reduced_play_log.pkl'
df_play = pd.read_pickle(play_file_pickle)

In [3]:
df_play.head()

,index,uid,device,song_type,play_time,song_length,label,date,song_id_new,singer_popularity_rank
0,0,154824972,ar,0,22.0,178,1,20170302,311464,7468
1,1,154412619,ar,0,235.0,235,1,20170302,19974,17
2,2,154830959,ar,0,256.0,256,1,20170302,116,11
3,3,154826319,ar,0,27962.0,0,1,20170302,175551,141
4,4,154470406,ar,0,5.0,247,1,20170302,411,187


In [4]:
# For churn prediction, choose the logs before the cut-off date 20170422
df_play = df_play.loc[df_play.date < '20170422']

In [5]:
df_play.date.unique()

array(['20170302', '20170303', '20170304', '20170305', '20170306',
       '20170307', '20170308', '20170309', '20170330', '20170331',
       '20170339', '20170401', '20170402', '20170403', '20170404',
       '20170405', '20170406', '20170407', '20170408', '20170409',
       '20170410', '20170411', '20170412', '20170413', '20170414',
       '20170415', '20170416', '20170417', '20170418', '20170419',
       '20170420', '20170421', '20170301'], dtype=object)

### Two features to create: 1. rare_song_player (taking 0,1), and 2. song_popular_ratio

#### Get the set of most popular songs

In [6]:
number_of_times_played_dist = df_play[['uid','song_id_new']].groupby('song_id_new').count()\
                            .rename(columns={"uid": "play_count"})
                              

In [7]:
number_of_times_played_dist.reset_index(inplace = True)

In [8]:
number_of_times_played_dist.head()

,song_id_new,play_count
0,1,69782
1,2,55781
2,3,40590
3,4,32097
4,5,29086


In [9]:
number_of_times_played_dist.tail()

,song_id_new,play_count
515348,618477,1
515349,618478,1
515350,618479,1
515351,618480,1
515352,618481,1


In [10]:
bins = [0, 1, 100, 1000, 10000, 100000]

In [11]:
group_names = ['1','1~100', '100~1000', '1000~10000', '10000~10000']

In [12]:
number_of_times_played_dist['group'] = pd.cut(number_of_times_played_dist.play_count, bins, labels=group_names)

In [13]:
number_of_times_played_dist.group.value_counts()

1~100          306083
1              197181
100~1000        11103
1000~10000        964
10000~10000        22
Name: group, dtype: int64

In [14]:
# Calculate the percentage of each group
number_of_times_played_dist.group.value_counts()/number_of_times_played_dist.shape[0]

1~100          0.593929
1              0.382613
100~1000       0.021544
1000~10000     0.001871
10000~10000    0.000043
Name: group, dtype: float64

#### Most songs are played less than 100 times, thus we could set songs played more than 1000 times as popular songs, top 0.25%.

In [15]:
# Popular songs:
songs_id_popular = set(number_of_times_played_dist\
                    [number_of_times_played_dist['group'].isin(['1000~10000','10000~10000'])]\
                    .song_id_new)

In [16]:
len(songs_id_popular)

986

In [17]:
df_play['song_popular'] = df_play.song_id_new.isin(songs_id_popular)
df_play.head()

,index,uid,device,song_type,play_time,song_length,label,date,song_id_new,singer_popularity_rank,song_popular
0,0,154824972,ar,0,22.0,178,1,20170302,311464,7468,False
1,1,154412619,ar,0,235.0,235,1,20170302,19974,17,False
2,2,154830959,ar,0,256.0,256,1,20170302,116,11,True
3,3,154826319,ar,0,27962.0,0,1,20170302,175551,141,False
4,4,154470406,ar,0,5.0,247,1,20170302,411,187,True


In [18]:
# Make a new dataframe with each uid summarized behavior as one column
df = pd.DataFrame(columns= ['uid'])

In [19]:
df.uid = df_play.groupby(['uid']).size().index

In [20]:
df.head()

,uid
0,100077577
1,100419770
2,100474444
3,100751531
4,101231687


In [21]:
df_song_popular = df_play[['uid','song_popular']].groupby('uid').mean().reset_index()

In [22]:
df_song_popular.head()

,uid,song_popular
0,100077577,0.176471
1,100419770,0.113208
2,100474444,0.579710
3,100751531,0.346154
4,101231687,0.197889


In [23]:
df = df.merge(df_song_popular, left_on = 'uid', right_on = 'uid', how = 'inner')
df.head()

,uid,song_popular
0,100077577,0.176471
1,100419770,0.113208
2,100474444,0.579710
3,100751531,0.346154
4,101231687,0.197889


In [24]:
#majority vote of device
df_device = df_play[['uid', 'device']].groupby('uid')['device'].apply(lambda x: x.value_counts().index[0])
df_device.head()

uid
100077577     ip
100419770     ar
100474444     ar
100751531     ar
101231687     ar
Name: device, dtype: object

In [25]:
df_device = df_device.reset_index()

In [26]:
df_device.head()

,uid,device
0,100077577,ip
1,100419770,ar
2,100474444,ar
3,100751531,ar
4,101231687,ar


In [27]:
df_device.device.value_counts()

ar    63475
ip    10019
Name: device, dtype: int64

In [28]:
# majority vote of song_type
df_song_type = df_play[['uid', 'song_type']].groupby('uid')['song_type']\
                .apply(lambda x: x.value_counts().index[0]).reset_index()
df_song_type.head()

,uid,song_type
0,100077577,0
1,100419770,1
2,100474444,0
3,100751531,1
4,101231687,0


In [29]:
df_song_type.song_type.value_counts()

0    67414
1     5490
2      588
3        2
Name: song_type, dtype: int64

In [30]:
# Merge device into df
df = df.merge(df_device, left_on = 'uid', right_on = 'uid', how = 'inner')
df.head()

,uid,song_popular,device
0,100077577,0.176471,ip
1,100419770,0.113208,ar
2,100474444,0.579710,ar
3,100751531,0.346154,ar
4,101231687,0.197889,ar


In [31]:
# Merge song_type into df
df = df.merge(df_song_type, left_on = 'uid', right_on = 'uid', how = 'inner')
df.head()

,uid,song_popular,device,song_type
0,100077577,0.176471,ip,0
1,100419770,0.113208,ar,1
2,100474444,0.579710,ar,0
3,100751531,0.346154,ar,1
4,101231687,0.197889,ar,0


In [32]:
df = df.rename(columns={'song_type' : 'major_song_type'})

In [33]:
df_song_popularity = df_play[['uid','song_popular']].groupby('uid')\
                    .mean().rename(columns = {'song_popular' : 'song_popularity_ratio'})


In [34]:
df_song_popularity = df_song_popularity.reset_index()
df_song_popularity.head()

,uid,song_popularity_ratio
0,100077577,0.176471
1,100419770,0.113208
2,100474444,0.579710
3,100751531,0.346154
4,101231687,0.197889


In [35]:
# Merge song_popularity_ratio into df
df = df.merge(df_song_popularity, left_on = 'uid', right_on = 'uid', how = 'inner')
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio
0,100077577,0.176471,ip,0,0.176471
1,100419770,0.113208,ar,1,0.113208
2,100474444,0.579710,ar,0,0.579710
3,100751531,0.346154,ar,1,0.346154
4,101231687,0.197889,ar,0,0.197889


In [36]:
# Total Play_time
df_total_play_time = df_play[['uid','play_time']].groupby('uid').sum()/60# change time unit to minutes


In [37]:
df_total_play_time = df_total_play_time.rename(columns = {'play_time':'total_play_time'}).reset_index()

In [38]:
df_total_play_time.head()

,uid,total_play_time
0,100077577,3.466667
1,100419770,115335.033333
2,100474444,113.666667
3,100751531,42.183333
4,101231687,2962.783333


In [39]:
sum(df_total_play_time.total_play_time>100000)

642

In [40]:
df = df.merge(df_total_play_time, left_on = 'uid', right_on = 'uid', how = 'inner')
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time
0,100077577,0.176471,ip,0,0.176471,3.466667
1,100419770,0.113208,ar,1,0.113208,115335.033333
2,100474444,0.579710,ar,0,0.579710,113.666667
3,100751531,0.346154,ar,1,0.346154,42.183333
4,101231687,0.197889,ar,0,0.197889,2962.783333


In [41]:

df_play[['label']] = df_play[['label']].astype('int')
df_label = df_play[['uid','label']].groupby('uid').mean().reset_index()
df_label.head()

,uid,label
0,100077577,1
1,100419770,0
2,100474444,1
3,100751531,1
4,101231687,0


In [42]:
# Merge label into df
df = df.merge(df_label, left_on = 'uid', right_on= 'uid', how = 'inner')
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label
0,100077577,0.176471,ip,0,0.176471,3.466667,1
1,100419770,0.113208,ar,1,0.113208,115335.033333,0
2,100474444,0.579710,ar,0,0.579710,113.666667,1
3,100751531,0.346154,ar,1,0.346154,42.183333,1
4,101231687,0.197889,ar,0,0.197889,2962.783333,0


In [43]:
# Total play_time
# Create bins for total_play_time
bins = [-1,100,200,300,400,500,600,700,800,900,1000,2000,3000,4000,5000,6000,7000,8000,9000,10000,float('inf')]
bin_names = ['0-100','100-200','200-300','300-400','400-500','500-600','600-700','700-800','800-900','900-1k',\
             '1k-2k','2k-3k','3k-4k','4k-5k','5k-6k','6k-7k','7k-8k','8k-9k','9k-10k','10k+']
df['play_time_label'] = pd.cut(df.total_play_time,bins, labels = bin_names)
df.play_time_label.value_counts()

0-100      41702
100-200     9133
200-300     4652
300-400     3162
1k-2k       3124
400-500     2276
500-600     1732
10k+        1650
600-700     1398
700-800     1159
2k-3k        939
800-900      911
900-1k       691
3k-4k        386
4k-5k        201
5k-6k        134
6k-7k         87
7k-8k         73
9k-10k        34
8k-9k         34
Name: play_time_label, dtype: int64

In [44]:
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label
0,100077577,0.176471,ip,0,0.176471,3.466667,1,0-100
1,100419770,0.113208,ar,1,0.113208,115335.033333,0,10k+
2,100474444,0.579710,ar,0,0.579710,113.666667,1,100-200
3,100751531,0.346154,ar,1,0.346154,42.183333,1,0-100
4,101231687,0.197889,ar,0,0.197889,2962.783333,0,2k-3k


In [45]:
df[['play_time_label', 'label']].groupby('play_time_label').mean()

,label
play_time_label,
0-100,0.807467
100-200,0.632541
200-300,0.546217
300-400,0.498102
400-500,0.481107
500-600,0.411663
600-700,0.389127
700-800,0.339948
800-900,0.346872


#### There is definately dependence between churn rate and total_play_time. <br> And the churn rate is similar for total_play_time = (100, 400], (400, 700], (700, 1k], (1k, 7k], (7k, 10k], (10k+] 

In [46]:
df = df.drop(['play_time_label'], axis = 1)

In [47]:
# Reduce number of total_play_time bins. 
bins = [-1,100,400,700,1000,7000,10000,float('inf')]
bin_names = ['0-100','100-400','400-700','700-1k','1k-7k','7k-10k','10k+']
df['play_time_label'] = pd.cut(df.total_play_time,bins, labels = bin_names)
df.play_time_label.value_counts()

0-100      41702
100-400    16947
400-700     5406
1k-7k       4871
700-1k      2761
10k+        1650
7k-10k       141
Name: play_time_label, dtype: int64

In [48]:
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label
0,100077577,0.176471,ip,0,0.176471,3.466667,1,0-100
1,100419770,0.113208,ar,1,0.113208,115335.033333,0,10k+
2,100474444,0.579710,ar,0,0.579710,113.666667,1,100-400
3,100751531,0.346154,ar,1,0.346154,42.183333,1,0-100
4,101231687,0.197889,ar,0,0.197889,2962.783333,0,1k-7k


In [49]:
df[['play_time_label', 'label']].groupby('play_time_label').mean()

,label
play_time_label,
0-100,0.807467
100-400,0.583761
400-700,0.435072
700-1k,0.330677
1k-7k,0.271197
7k-10k,0.496454
10k+,0.815152


In [50]:
# Average play_time
df_avg_play_time = df_play[['uid', 'play_time']].\
                    groupby('uid').mean().rename(columns = {'play_time' : 'avg_play_time'}).reset_index()

In [51]:
df_avg_play_time.head()

,uid,avg_play_time
0,100077577,12.235294
1,100419770,130567.962264
2,100474444,98.840580
3,100751531,97.346154
4,101231687,234.521108


In [52]:
# Merge avg_play_time into df
df = df.merge(df_avg_play_time, left_on = 'uid', right_on = 'uid', how = 'inner')

In [53]:
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time
0,100077577,0.176471,ip,0,0.176471,3.466667,1,0-100,12.235294
1,100419770,0.113208,ar,1,0.113208,115335.033333,0,10k+,130567.962264
2,100474444,0.579710,ar,0,0.579710,113.666667,1,100-400,98.840580
3,100751531,0.346154,ar,1,0.346154,42.183333,1,0-100,97.346154
4,101231687,0.197889,ar,0,0.197889,2962.783333,0,1k-7k,234.521108


In [54]:
# Create a set for the rarely played songs, which are played once in the entire dataframe
num_song_least_pop = sum(df_play.groupby('song_id_new').size()<=1)
num_song_least_pop

197181

In [55]:
least_popular_songs = df_play['song_id_new'].value_counts()[-1:-num_song_least_pop-1:-1].index

In [56]:
# Create a new column in original df_play to indicate whether the played song is in the least popular set.
df_play['least_popular'] = df_play.song_id_new.isin(set(least_popular_songs))

In [57]:
# For each uid, count the number of times played the least popular song
df_least_pop_counts = df_play[['uid', 'least_popular']].groupby('uid').\
                        sum().rename(columns = {'least_popular' : 'least_pop_count'}).reset_index()

In [58]:
df_least_pop_counts.head()

,uid,least_pop_count
0,100077577,0.0
1,100419770,1.0
2,100474444,0.0
3,100751531,4.0
4,101231687,0.0


In [59]:
# Merge the least_pop_count into df
df = df.merge(df_least_pop_counts, left_on = 'uid', right_on = 'uid', how = 'inner')

In [60]:
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time,least_pop_count
0,100077577,0.176471,ip,0,0.176471,3.466667,1,0-100,12.235294,0.0
1,100419770,0.113208,ar,1,0.113208,115335.033333,0,10k+,130567.962264,1.0
2,100474444,0.579710,ar,0,0.579710,113.666667,1,100-400,98.840580,0.0
3,100751531,0.346154,ar,1,0.346154,42.183333,1,0-100,97.346154,4.0
4,101231687,0.197889,ar,0,0.197889,2962.783333,0,1k-7k,234.521108,0.0


In [61]:
df.least_pop_count.value_counts()

0.0      45219
1.0       8778
2.0       4705
3.0       2976
4.0       2088
5.0       1521
6.0       1150
7.0        935
8.0        748
9.0        592
10.0       467
11.0       402
12.0       381
13.0       322
15.0       252
14.0       248
16.0       203
17.0       174
18.0       169
20.0       149
19.0       138
21.0       125
23.0       111
22.0        98
24.0        96
25.0        76
26.0        74
28.0        69
27.0        69
29.0        64
         ...  
699.0        1
139.0        1
219.0        1
233.0        1
160.0        1
108.0        1
127.0        1
399.0        1
283.0        1
175.0        1
147.0        1
575.0        1
242.0        1
161.0        1
171.0        1
454.0        1
335.0        1
158.0        1
483.0        1
179.0        1
178.0        1
744.0        1
289.0        1
150.0        1
572.0        1
169.0        1
241.0        1
129.0        1
214.0        1
142.0        1
Name: least_pop_count, Length: 188, dtype: int64

In [62]:
# Add a column in df to show the total play count
df['total_play_count'] = list(df_play.groupby('uid').size())

In [63]:
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time,least_pop_count,total_play_count
0,100077577,0.176471,ip,0,0.176471,3.466667,1,0-100,12.235294,0.0,17
1,100419770,0.113208,ar,1,0.113208,115335.033333,0,10k+,130567.962264,1.0,53
2,100474444,0.579710,ar,0,0.579710,113.666667,1,100-400,98.840580,0.0,69
3,100751531,0.346154,ar,1,0.346154,42.183333,1,0-100,97.346154,4.0,26
4,101231687,0.197889,ar,0,0.197889,2962.783333,0,1k-7k,234.521108,0.0,758


In [64]:
df['least_popular_ratio'] = df.apply(lambda x: x.least_pop_count/x.total_play_count, axis = 1)
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time,least_pop_count,total_play_count,least_popular_ratio
0,100077577,0.176471,ip,0,0.176471,3.466667,1,0-100,12.235294,0.0,17,0.000000
1,100419770,0.113208,ar,1,0.113208,115335.033333,0,10k+,130567.962264,1.0,53,0.018868
2,100474444,0.579710,ar,0,0.579710,113.666667,1,100-400,98.840580,0.0,69,0.000000
3,100751531,0.346154,ar,1,0.346154,42.183333,1,0-100,97.346154,4.0,26,0.153846
4,101231687,0.197889,ar,0,0.197889,2962.783333,0,1k-7k,234.521108,0.0,758,0.000000


In [65]:
# Examine the dependence of churn rate on playing least popular songs:
df.groupby(pd.cut(df.least_popular_ratio,\
          np.percentile(df.least_popular_ratio,[0,64,80,90,100]), include_lowest=True, ))\
          .mean()['label']

# The dependence of churn-rate on least_popular_ratio is not clear, yet...

least_popular_ratio
(-0.001, 0.00452]    0.726598
(0.00452, 0.0395]    0.459942
(0.0395, 0.111]      0.613932
(0.111, 1.0]         0.751272
Name: label, dtype: float64

#### Examine the cutoff number of play logs for labeling popular songs: 100, 500,1000,2000

In [66]:
popular_songs_cutoff = [100,500,1000,2000]
cutoff_quantile_dict = {}
for i in range(len(popular_songs_cutoff)):
    print 'procesing no.%d cutoff: %d' % (i,popular_songs_cutoff[i])
    num_pop_songs = sum(df_play.groupby('song_id_new').size()>popular_songs_cutoff[i])
    popular_songs = df_play['song_id_new'].value_counts()[:(num_pop_songs-1)].index
    df_play['is_popular'] = df_play.song_id_new.isin(set(popular_songs))
    df_temp = df_play.groupby('uid').mean()
    cutoff_quantile_dict[i] = df_temp.groupby(pd.cut(df_temp.is_popular, \
                           np.percentile(df_temp.is_popular, [0,25,50,75,100]),include_lowest = True)).mean()['label']
cutoff_quantile_dict

procesing no.0 cutoff: 100
procesing no.1 cutoff: 500
procesing no.2 cutoff: 1000
procesing no.3 cutoff: 2000


{0: is_popular
 (-0.001, 0.413]    0.685806
 (0.413, 0.697]     0.603766
 (0.697, 0.917]     0.609742
 (0.917, 1.0]       0.800912
 Name: label, dtype: float64, 1: is_popular
 (-0.001, 0.166]    0.693333
 (0.166, 0.403]     0.614020
 (0.403, 0.675]     0.628749
 (0.675, 1.0]       0.762410
 Name: label, dtype: float64, 2: is_popular
 (-0.001, 0.0833]    0.706147
 (0.0833, 0.276]     0.599672
 (0.276, 0.527]      0.631144
 (0.527, 1.0]        0.761117
 Name: label, dtype: float64, 3: is_popular
 (-0.001, 0.0204]    0.742887
 (0.0204, 0.154]     0.574504
 (0.154, 0.352]      0.633644
 (0.352, 1.0]        0.747796
 Name: label, dtype: float64}

In [67]:
pd.DataFrame({popular_songs_cutoff[0]:list(cutoff_quantile_dict[0]),
             popular_songs_cutoff[1]:list(cutoff_quantile_dict[1]),
             popular_songs_cutoff[2]:list(cutoff_quantile_dict[2]),
             popular_songs_cutoff[3]:list(cutoff_quantile_dict[3])})

,100,500,1000,2000
0,0.685806,0.693333,0.706147,0.742887
1,0.603766,0.614020,0.599672,0.574504
2,0.609742,0.628749,0.631144,0.633644
3,0.800912,0.762410,0.761117,0.747796


The different cutoffs give similar results:

    When the user does play popular songs a lot( pop ratio in the >75% quantile among all users), the churn rate
    become higher.
But from the table above, it seems the mean popular music play counts does not seem to easily distinguish churned users from loyal users. Now check the accumulated play_counts of popular music. 

In [68]:
popular_songs_cutoff = [100,500,1000,2000]
cutoff_quantile_dict = {}
for i in range(len(popular_songs_cutoff)):
    print 'procesing no.%d cutoff: %d' % (i,popular_songs_cutoff[i])
    num_pop_songs = sum(df_play.groupby('song_id_new').size()>popular_songs_cutoff[i])
    popular_songs = df_play['song_id_new'].value_counts()[:(num_pop_songs-1)].index
    df_play['is_popular'] = df_play.song_id_new.isin(set(popular_songs))
    df_temp = df_play[['uid', 'is_popular','label']].groupby('uid').\
              agg({'is_popular': ['sum'], 'label':['mean']}).\
              rename(columns = {'is_popular': 'is_popular_count', 'label': 'label'})
    df_temp.columns = df_temp.columns.get_level_values(0)
    df_temp.label = df_temp.label.apply(lambda x: x>0)
    cutoff_quantile_dict[i] = df_temp.groupby(pd.cut(df_temp.is_popular_count, \
                           np.percentile(df_temp.is_popular_count, [0,25,50,75,100]),include_lowest = True)).\
                           mean()['label']
cutoff_quantile_dict

procesing no.0 cutoff: 100
procesing no.1 cutoff: 500
procesing no.2 cutoff: 1000
procesing no.3 cutoff: 2000


{0: is_popular_count
 (-0.001, 7.0]      0.830621
 (7.0, 20.0]        0.798866
 (20.0, 70.0]       0.663383
 (70.0, 30174.0]    0.399530
 Name: label, dtype: float64, 1: is_popular_count
 (-0.001, 3.0]      0.800799
 (3.0, 13.0]        0.789791
 (13.0, 43.0]       0.683552
 (43.0, 30135.0]    0.415696
 Name: label, dtype: float64, 2: is_popular_count
 (-0.001, 2.0]      0.787194
 (2.0, 9.0]         0.778708
 (9.0, 31.0]        0.699371
 (31.0, 30133.0]    0.430310
 Name: label, dtype: float64, 3: is_popular_count
 (-0.001, 1.0]      0.777203
 (1.0, 6.0]         0.762127
 (6.0, 20.0]        0.694892
 (20.0, 30113.0]    0.446988
 Name: label, dtype: float64}

When using 1000 as the cutoff play counts for popular songs, the 25% and 75% quantiles of users' popular
song values, 6 and 21 can differentiate the users significantly. Say n=played_times:

    When n < 6, churn possibility is high.

    When n > 21, churn possibility is low.

When using 100 as the cutoff play counts,25%, 75% quantile cutoffs are 7 and 72 for defining popular songs as played more than 100 times. Say n=played_times:

    When n < 7, churn possibility is high.

    When n > 72, churn possibility is low.
Looks like 100 works the best as the last quantile gives the smallest churn-rate.

In [69]:
# cutoff value for popular song play count is 100
popular_song_cutoff = 100
popular_songs_num = sum(df_play.groupby('song_id_new').size() > popular_song_cutoff)
popular_songs = df_play['song_id_new'].value_counts()[:(popular_songs_num-1)].index
df_play['is_popular'] = df_play.song_id_new.isin(set(popular_songs))


In [70]:
df_temp = df_play[['uid','is_popular','label']].groupby('uid').sum()
df_temp.label = df_temp.label.apply(lambda x: int(x>0))
df_temp.groupby(pd.cut(df_temp.is_popular, \
                           np.percentile(df_temp.is_popular, [0, 25,75, 100]),include_lowest = True)).mean()['label']

is_popular
(-0.001, 7.0]      0.830621
(7.0, 70.0]        0.728731
(70.0, 30174.0]    0.399530
Name: label, dtype: float64

In [71]:
df['most_popular_count'] = list(df_play.groupby('uid').sum()['is_popular'])
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time,least_pop_count,total_play_count,least_popular_ratio,most_popular_count
0,100077577,0.176471,ip,0,0.176471,3.466667,1,0-100,12.235294,0.0,17,0.000000,10.0
1,100419770,0.113208,ar,1,0.113208,115335.033333,0,10k+,130567.962264,1.0,53,0.018868,15.0
2,100474444,0.579710,ar,0,0.579710,113.666667,1,100-400,98.840580,0.0,69,0.000000,67.0
3,100751531,0.346154,ar,1,0.346154,42.183333,1,0-100,97.346154,4.0,26,0.153846,13.0
4,101231687,0.197889,ar,0,0.197889,2962.783333,0,1k-7k,234.521108,0.0,758,0.000000,539.0


#### Examine the count of plays in different time window


In [72]:
df_daily_count = df_play.groupby(['uid','date']).size().unstack().fillna(0)

In [73]:
df_daily_count.head()

date,20170301,20170302,20170303,20170304,20170305,20170306,20170307,20170308,20170309,20170330,...,20170412,20170413,20170414,20170415,20170416,20170417,20170418,20170419,20170420,20170421
uid,,,,,,,,,,,,,,,,,,,,,
100077577,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100419770,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100474444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100751531,16.0,0.0,3.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101231687,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,105.0,...,29.0,32.0,0.0,0.0,0.0,0.0,80.0,18.0,0.0,32.0


In [74]:
days = df_daily_count.shape[1]
days

33

#### Divide the 33 days into 5 time window, each window is approximately 1 week.
(0,9], (9,12], (12,19], (19,26], (26, 33] are the days windows from the cut-off date 2017/04/22.

In [75]:
day_windows = [days, 26, 19, 12, 9, 0]


In [80]:
for i in xrange(len(day_windows)-1): 
    for j in range(day_windows[i+1],day_windows[i]):
        print j,
    print
        

26 27 28 29 30 31 32 0
19 20 21 22 23 24 25 1
12 13 14 15 16 17 18 2
9 10 11 3
0 1 2 3 4 5 6 7 8 4


In [85]:
for i in xrange(len(day_windows)-1):
    if i == len(day_windows)-2:
        colname = 'count_in_recent_'+str(i+4)+'week'
    else:
        colname = 'count_in_recent_'+str(i+1)+'week'
    df[colname] = list(df_daily_count.iloc[:,day_windows[i+1]:day_windows[i]].sum(axis = 1).astype('int'))
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,label,play_time_label,avg_play_time,least_pop_count,total_play_count,least_popular_ratio,most_popular_count,count_in_recent_1week,count_in_recent_2week,count_in_recent_3week,count_in_recent_4week,count_in_recent_8week
0,100077577,0.176471,ip,0,0.176471,3.466667,1,0-100,12.235294,0.0,17,0.000000,10.0,0,0,0,17,0
1,100419770,0.113208,ar,1,0.113208,115335.033333,0,10k+,130567.962264,1.0,53,0.018868,15.0,0,7,45,1,0
2,100474444,0.579710,ar,0,0.579710,113.666667,1,100-400,98.840580,0.0,69,0.000000,67.0,0,0,0,69,0
3,100751531,0.346154,ar,1,0.346154,42.183333,1,0-100,97.346154,4.0,26,0.153846,13.0,0,0,0,0,26
4,101231687,0.197889,ar,0,0.197889,2962.783333,0,1k-7k,234.521108,0.0,758,0.000000,539.0,130,84,403,141,0


In [86]:
df.shape

(73494, 18)

In [87]:
labels = list(df['label'])
df = df.drop(['label','least_popular_ratio','total_play_count'],axis = 1)
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,play_time_label,avg_play_time,least_pop_count,most_popular_count,count_in_recent_1week,count_in_recent_2week,count_in_recent_3week,count_in_recent_4week,count_in_recent_8week
0,100077577,0.176471,ip,0,0.176471,3.466667,0-100,12.235294,0.0,10.0,0,0,0,17,0
1,100419770,0.113208,ar,1,0.113208,115335.033333,10k+,130567.962264,1.0,15.0,0,7,45,1,0
2,100474444,0.579710,ar,0,0.579710,113.666667,100-400,98.840580,0.0,67.0,0,0,0,69,0
3,100751531,0.346154,ar,1,0.346154,42.183333,0-100,97.346154,4.0,13.0,0,0,0,0,26
4,101231687,0.197889,ar,0,0.197889,2962.783333,1k-7k,234.521108,0.0,539.0,130,84,403,141,0


In [88]:
df['label'] = labels
df.label = df.label.astype(int)
df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,play_time_label,avg_play_time,least_pop_count,most_popular_count,count_in_recent_1week,count_in_recent_2week,count_in_recent_3week,count_in_recent_4week,count_in_recent_8week,label
0,100077577,0.176471,ip,0,0.176471,3.466667,0-100,12.235294,0.0,10.0,0,0,0,17,0,1
1,100419770,0.113208,ar,1,0.113208,115335.033333,10k+,130567.962264,1.0,15.0,0,7,45,1,0,0
2,100474444,0.579710,ar,0,0.579710,113.666667,100-400,98.840580,0.0,67.0,0,0,0,69,0,1
3,100751531,0.346154,ar,1,0.346154,42.183333,0-100,97.346154,4.0,13.0,0,0,0,0,26,1
4,101231687,0.197889,ar,0,0.197889,2962.783333,1k-7k,234.521108,0.0,539.0,130,84,403,141,0,0


In [89]:
df.shape

(73494, 16)

#### Save dataframe as pkl

In [90]:
filename_pickle = '/Users/Xiaoxi/Desktop/BitTiger/Capstone/data/output/play_full_features_and_label.pkl'
df.to_pickle(filename_pickle)

In [91]:
# test df is saved successfully
test_df = pd.read_pickle(filename_pickle)
test_df.shape

(73494, 16)

In [92]:
test_df.head()

,uid,song_popular,device,major_song_type,song_popularity_ratio,total_play_time,play_time_label,avg_play_time,least_pop_count,most_popular_count,count_in_recent_1week,count_in_recent_2week,count_in_recent_3week,count_in_recent_4week,count_in_recent_8week,label
0,100077577,0.176471,ip,0,0.176471,3.466667,0-100,12.235294,0.0,10.0,0,0,0,17,0,1
1,100419770,0.113208,ar,1,0.113208,115335.033333,10k+,130567.962264,1.0,15.0,0,7,45,1,0,0
2,100474444,0.579710,ar,0,0.579710,113.666667,100-400,98.840580,0.0,67.0,0,0,0,69,0,1
3,100751531,0.346154,ar,1,0.346154,42.183333,0-100,97.346154,4.0,13.0,0,0,0,0,26,1
4,101231687,0.197889,ar,0,0.197889,2962.783333,1k-7k,234.521108,0.0,539.0,130,84,403,141,0,0
